In [27]:
from transformers import BertTokenizer, BertPreTrainedModel, AdamW, AutoTokenizer, BertConfig, BertModel
from rbert_model import RBERT
import os
from rbert_data_loader import load_and_cache_examples
from train_relation_extraction import RelationExtractorTrainer, get_tokenizer
from rbert_data_loader import TermFrameProcessor, convert_examples_to_features

In [14]:
import torch


In [8]:
device = torch.device('cuda')

In [9]:
device

device(type='cuda')

In [20]:
conf = {'experiment': 'EN_reg_nonhier+def',
        'model_name': 'Scibert-cased_00',
        'tokenizer_id': 'Scibert-cased',
        'model_id': 'Scibert-cased',
        'max_length': 128,
        'batch_size': 4,
        'epochs': 5}
conf['model_dir'] = os.path.join('data', 'experiments', conf['experiment'], conf['model_name'])
conf['eval_dir'] = conf['model_dir']
conf['data_dir'] = os.path.join('data', 'experiments', conf['experiment'])

In [22]:
tokenizer = get_tokenizer(conf['tokenizer_id'])

In [24]:
processor = TermFrameProcessor(conf)

In [ ]:
args = torch.load(os.path.join(conf['model_dir'], "training_args.bin"))
model = RBERT.from_pretrained(os.path.join(conf['model_dir'], 'model.pt'), args=args)
model.to(device);

In [79]:
lines = [['Other', "i'm doing nothing <e1> right now </e1>, do you want to <e2> come to me </e2>?"]]

In [80]:
examples = processor._create_examples(lines, 'train')

In [81]:
features = convert_examples_to_features(
            examples, conf['max_length'], tokenizer, add_sep_token=False
        )

In [82]:
input_ids = torch.Tensor([features[0].input_ids]).type(torch.long).to(device)
attention_mask = torch.Tensor([features[0].attention_mask]).type(torch.long).to(device)
token_type_ids = torch.Tensor([features[0].token_type_ids]).type(torch.long).to(device)
labels = None
e1_mask = torch.Tensor([features[0].e1_mask]).type(torch.long).to(device)
e2_mask = torch.Tensor([features[0].e2_mask]).type(torch.long).to(device)

In [75]:
model.eval();

In [83]:
model(input_ids, attention_mask, token_type_ids, labels, e1_mask, e2_mask)

(tensor([[-2.7069, -0.5903,  3.5867, -0.5823, -1.7594,  2.9533, -1.6908]],
        device='cuda:0', grad_fn=<AddmmBackward>),)